### Выполнила: Ковалева Екатерина Сергеевна  НИЯУ МИФИ группа М24-525

## Импорты

In [58]:
import pandas as pd
import numpy as np
import random
import time
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD, accuracy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


## Сид и девайс

In [46]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [47]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {DEVICE}")

Используется устройство: cpu


## Небольшая предобработка данных

In [48]:
data = pd.read_parquet('/home/administrator/Desktop/IDE/Recomendation/data_advanced.parquet')

In [49]:
def tags_to_string(tags):
    if isinstance(tags, np.ndarray):
        return ' '.join(tags.astype(str))
    elif pd.isna(tags):
        return ''
    else:
        return str(tags)

data['tags_list_str'] = data['tags_list'].apply(tags_to_string)

In [51]:
# Маппинг ID → индексы
user_ids = data['user_id'].unique()
book_ids = data['book_id'].unique()
USER_COUNT = len(user_ids)
BOOK_COUNT = len(book_ids)

user_to_index = {uid: i for i, uid in enumerate(user_ids)}
book_to_index = {bid: i for i, bid in enumerate(book_ids)}

data['user_index'] = data['user_id'].map(user_to_index)
data['book_index'] = data['book_id'].map(book_to_index)


## Разделение на треин и тест

In [52]:
train, test = sklearn_train_test_split(data, test_size=0.2, stratify=(data['rating'] >= 3.5), random_state=42)
all_items = data['book_id'].unique()
user_seen_items = train.groupby('user_id')['book_id'].agg(set).to_dict()
print(f"Train: {len(train):,}, Test: {len(test):,}")

Train: 4,781,183, Test: 1,195,296


## Полезные функции

In [50]:
def evaluate_metrics(recommender, test_data, user_seen_items_dict, all_items, k=10):
    test_user_items = test_data.groupby('user_id')['book_id'].agg(list).to_dict()
    precision_sum = recall_sum = ndcg_sum = 0
    users_evaluated = 0

    for user_id, true_items in tqdm(test_user_items.items(), desc=recommender.__class__.__name__):
        if not true_items:
            continue
        user_history = user_seen_items_dict.get(user_id, set())
        recs = recommender.recommend(user_id, user_history, k=k)
        if not recs:
            continue

        users_evaluated += 1
        rec_set = set(recs)
        true_set = set(true_items)
        hits = rec_set & true_set

        precision_sum += len(hits) / len(recs)
        recall_sum += len(hits) / len(true_items)

        relevance = [1 if item in true_set else 0 for item in recs]
        dcg = sum(r / np.log2(i + 2) for i, r in enumerate(relevance))
        ideal = sorted(relevance, reverse=True)
        idcg = sum(r / np.log2(i + 2) for i, r in enumerate(ideal))
        ndcg_sum += dcg / idcg if idcg > 0 else 0.0

    if users_evaluated == 0:
        return {'precision@10': 0, 'recall@10': 0, 'ndcg@10': 0}
    return {
        'precision@10': precision_sum / users_evaluated,
        'recall@10': recall_sum / users_evaluated,
        'ndcg@10': ndcg_sum / users_evaluated
    }

# Сегментация пользователей
user_ratings_count = train.groupby('user_id')['book_id'].count().to_dict()

def get_user_segment(user_id):
    n = user_ratings_count.get(user_id, 0)
    if n <= 0:
        return "cold"
    elif n <= 10:
        return "warm"
    else:
        return "active"

In [80]:
def diversity_rerank(recommendations, data, k=10, diversity_penalty=0.3):
    """
    Повышает разнообразие рекомендаций за счёт штрафа за общие теги.
    
    Параметры:
        recommendations: список (book_id, score)
        data: датафрейм с колонкой 'tags_list_str'
        k: итоговое число рекомендаций
        diversity_penalty: сила штрафа (0.0 = нет штрафа, 1.0 = полное подавление дублей)
    """
    if not recommendations:
        return []
    
    # Получаем теги для всех рекомендуемых книг
    book_tags = {}
    for book_id, _ in recommendations:
        tags_str = data[data['book_id'] == book_id]['tags_list_str'].values
        if len(tags_str) > 0:
            book_tags[book_id] = set(tags_str[0].split())
        else:
            book_tags[book_id] = set()
    
    # Re-ranking
    selected = []
    remaining = recommendations.copy()
    
    while len(selected) < k and remaining:
        # Выбираем книгу с максимальным (score - штраф)
        best_idx = -1
        best_score = -1
        
        for i, (book_id, score) in enumerate(remaining):
            # Считаем штраф: чем больше общих тегов с уже выбранными — тем выше штраф
            penalty = 0.0
            if selected:
                current_tags = book_tags[book_id]
                overlap = 0
                for sel_id in selected:
                    overlap += len(current_tags & book_tags[sel_id])
                penalty = diversity_penalty * (overlap / len(current_tags) if current_tags else 0)
            
            adjusted_score = score - penalty
            if adjusted_score > best_score:
                best_score = adjusted_score
                best_idx = i
        
        # Добавляем лучшую книгу
        selected_book = remaining.pop(best_idx)
        selected.append(selected_book)
    
    return selected

# Модели для гибридной системы

*Сравнительные метрики в конце нотубука*

## 1.Item-Based CF 

*Создание user-item матрицы*

In [ ]:
ratings_train = train[['user_id', 'book_id', 'rating']].copy()
user_item_matrix = ratings_train.pivot_table(index='user_id', columns='book_id', values='rating')
user_item_matrix.fillna(0, inplace=True)


*Расчёт схожести между книгами*

In [54]:
item_similarity = cosine_similarity(user_item_matrix.T)
item_sim_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

In [ ]:
class ItemBasedRecommender:
    def __init__(self, item_sim_df):
        self.item_sim_df = item_sim_df

    def recommend(self, user_id, user_seen_books, k=10):
        if not user_seen_books:
            return []
        seen_in_catalog = [bid for bid in user_seen_books if bid in self.item_sim_df.index]
        if not seen_in_catalog:
            return []
        similarity_scores = pd.Series(dtype='float64')
        for book_id in seen_in_catalog:
            sim_scores = self.item_sim_df[book_id]
            similarity_scores = similarity_scores.add(sim_scores, fill_value=0)
        similarity_scores = similarity_scores.drop(seen_in_catalog, errors='ignore')
        top_books = similarity_scores.nlargest(k)
        return top_books.index.tolist()

item_model = ItemBasedRecommender(item_sim_df)
print("Item-Based CF готова.")

Создание user-item матрицы для Item-Based CF...
Расчёт схожести между книгами...
✅ Item-Based CF готова.


## 2. Popularity модель (для cold-start)


In [55]:
class PopularityRecommender:
    def __init__(self, popular_books):
        self.popular_books = popular_books

    def recommend(self, user_id, user_seen_books, k=10):
        seen_set = set(user_seen_books)
        recs = [b for b in self.popular_books if b not in seen_set]
        return recs[:k]

pop_model = PopularityRecommender(train['book_id'].value_counts().index.tolist())
print("Popularity модель готова.")

Popularity модель готова.


## 3. SVD модель


In [62]:
from surprise import Dataset, Reader, SVD
train_surprise = train[['user_id', 'book_id', 'rating']].copy()
train_surprise['user_id'] = train_surprise['user_id'].astype(str)
train_surprise['book_id'] = train_surprise['book_id'].astype(str)

reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(train_surprise, reader)
trainset = dataset.build_full_trainset()

svd_algo = SVD(n_factors=50, n_epochs=20, random_state=42)
svd_algo.fit(trainset)

In [63]:
class SVDRecommender:
    def __init__(self, svd_model, train_data, k_candidates=2000):
        self.svd_model = svd_model
        self.popular_books = train_data['book_id'].value_counts().index.tolist()[:k_candidates]
        self.known_users = set(train_data['user_id'])

    def recommend(self, user_id, user_seen_books, k=10):
        if user_id not in self.known_users:
            seen_set = set(user_seen_books)
            return [b for b in self.popular_books if b not in seen_set][:k]
        seen_set = set(user_seen_books)
        books_to_predict = [b for b in self.popular_books if b not in seen_set]
        if not books_to_predict:
            return []
        testset = [[str(user_id), str(b), 4.0] for b in books_to_predict]
        predictions = self.svd_model.test(testset)
        predictions.sort(key=lambda x: x.est, reverse=True)
        return [int(pred.iid) for pred in predictions[:k]]

svd_model = SVDRecommender(svd_algo, train)
print("SVD модель готова.")

SVD модель готова.


## 4. Two-Tower модель 

In [64]:
USER_FEATURES = ['user_avg_rating', 'user_ratings_count', 'rating_deviation_user']
ITEM_FEATURES = ['weighted_rating', 'rating_std_dev']

train_sample = train.sample(n=min(100000, len(train)), random_state=42)

In [ ]:
class RecDataset(Dataset):
    def __init__(self, df, user_features, item_features):
        self.df = df
        self.user_idx = torch.LongTensor(df['user_index'].values)
        self.book_idx = torch.LongTensor(df['book_index'].values)
        self.user_feat = self._normalize(df[user_features])
        self.item_feat = self._normalize(df[item_features])
        self.labels = torch.FloatTensor((df['rating'] >= 3.5).values)
    
    def _normalize(self, df_feat):
        df = df_feat.copy()
        for col in df.columns:
            min_v, max_v = df[col].min(), df[col].max()
            df[col] = (df[col] - min_v) / (max_v - min_v) if max_v > min_v else 0.5
        return torch.FloatTensor(df.values)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return self.user_idx[idx], self.book_idx[idx], self.user_feat[idx], self.item_feat[idx], self.labels[idx]

class TwoTowerModel(nn.Module):
    def __init__(self, n_users, n_books, emb_dim, u_feat_dim, i_feat_dim):
        super().__init__()
        self.u_emb = nn.Embedding(n_users, emb_dim)
        self.i_emb = nn.Embedding(n_books, emb_dim)
        self.u_net = nn.Sequential(nn.Linear(emb_dim + u_feat_dim, 32), nn.ReLU(), nn.Linear(32, emb_dim))
        self.i_net = nn.Sequential(nn.Linear(emb_dim + i_feat_dim, 32), nn.ReLU(), nn.Linear(32, emb_dim))
    
    def forward(self, u_idx, i_idx, u_feat, i_feat):
        u_vec = self.u_net(torch.cat([self.u_emb(u_idx), u_feat], dim=1))
        i_vec = self.i_net(torch.cat([self.i_emb(i_idx), i_feat], dim=1))
        return (u_vec * i_vec).sum(dim=1).unsqueeze(1)

train_ds = RecDataset(train_sample, USER_FEATURES, ITEM_FEATURES)
train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)

model = TwoTowerModel(USER_COUNT, BOOK_COUNT, 16, len(USER_FEATURES), len(ITEM_FEATURES)).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(2):
    model.train()
    total_loss = 0
    for batch_idx, (u_idx, b_idx, u_feat, i_feat, labels) in enumerate(train_loader):
        u_idx, b_idx, u_feat, i_feat, labels = (
            u_idx.to(DEVICE), b_idx.to(DEVICE), u_feat.to(DEVICE), i_feat.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
        )
        optimizer.zero_grad()
        loss = criterion(model(u_idx, b_idx, u_feat, i_feat), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

model.eval()
all_book_idx = torch.arange(BOOK_COUNT).to(DEVICE)
with torch.no_grad():
    dummy_item_feat = torch.zeros(BOOK_COUNT, len(ITEM_FEATURES)).to(DEVICE)
    item_embeddings = model.i_net(torch.cat([model.i_emb(all_book_idx), dummy_item_feat], dim=1)).cpu().numpy()

class TwoTowerRecommender:
    def __init__(self, model, item_embeddings, user_to_index, book_to_index, idx_to_book_id):
        self.model = model.eval()
        self.item_embeddings = item_embeddings
        self.user_to_index = user_to_index
        self.book_to_index = book_to_index
        self.idx_to_book_id = idx_to_book_id

    def recommend(self, user_id, user_seen_books, k=10):
        if user_id not in self.user_to_index:
            return []
        user_idx = self.user_to_index[user_id]
        user_idx_tensor = torch.tensor([user_idx], dtype=torch.long).to(DEVICE)
        with torch.no_grad():
            dummy_user_feat = torch.zeros(1, len(USER_FEATURES)).to(DEVICE)
            user_emb = self.model.u_net(torch.cat([self.model.u_emb(user_idx_tensor), dummy_user_feat], dim=1)).cpu().numpy().flatten()
        scores = self.item_embeddings.dot(user_emb)
        ranked_indices = np.argsort(scores)[::-1]
        recs = []
        seen_set = set(user_seen_books)
        for idx in ranked_indices:
            book_id = self.idx_to_book_id.get(idx)
            if book_id is not None and book_id not in seen_set:
                recs.append(book_id)
                if len(recs) >= k:
                    break
        return recs

idx_to_book_id = {idx: bid for bid, idx in book_to_index.items()}
tt_model = TwoTowerRecommender(model, item_embeddings, user_to_index, book_to_index, idx_to_book_id)
print("Two-Tower модель готова.")

✅ Two-Tower модель готова.


# Оценка  моделей


In [67]:
SAMPLE_SIZE = 10000  # или 5000 для ещё большей скорости
test_users = test['user_id'].unique()
if len(test_users) > SAMPLE_SIZE:
    sampled_users = random.sample(list(test_users), SAMPLE_SIZE)
    test_sampled = test[test['user_id'].isin(sampled_users)].copy()
else:
    test_sampled = test.copy()

print(f"Оценка на {len(test_sampled['user_id'].unique()):,} пользователях (из {len(test_users):,})")

# Оценка моделей
models = [
    ('Popularity', pop_model),
    ('Item-Based CF', item_model),
    ('SVD', svd_model),
    ('Two-Tower', tt_model)
]

metrics_list = []
for name, model in models:
    start = time.time()
    metrics = evaluate_metrics(model, test_sampled, user_seen_items, all_items, k=10)
    elapsed = time.time() - start
    metrics_list.append({
        'model': name,
        'Precision@10': metrics['precision@10'],
        'Recall@10': metrics['recall@10'],
        'nDCG@10': metrics['ndcg@10']
    })
    print(f"{name}: nDCG@10 = {metrics['ndcg@10']:.4f}, время = {elapsed:.1f} сек")

Оценка на 10,000 пользователях (из 53,424)


PopularityRecommender: 100%|██████████| 10000/10000 [00:15<00:00, 652.95it/s]


Popularity: nDCG@10 = 0.3183, время = 15.5 сек


ItemBasedRecommender: 100%|██████████| 10000/10000 [08:17<00:00, 20.09it/s]


Item-Based CF: nDCG@10 = 0.6275, время = 498.0 сек


SVDRecommender: 100%|██████████| 10000/10000 [03:47<00:00, 43.95it/s]


SVD: nDCG@10 = 0.1097, время = 227.8 сек


TwoTowerRecommender: 100%|██████████| 10000/10000 [03:44<00:00, 44.60it/s]

Two-Tower: nDCG@10 = 0.0045, время = 224.5 сек


In [69]:
metrics_df = pd.DataFrame(metrics_list)

In [73]:
metrics_df

,model,Precision@10,Recall@10,nDCG@10
0,Popularity,0.09912,0.044831,0.318328
1,Item-Based CF,0.26869,0.126471,0.627471
2,SVD,0.02622,0.011746,0.109713
3,Two-Tower,0.00103,0.000480,0.004497


# Гибридная система

## гибрид 1. Попробовать все.

In [ ]:
def weights_from_metrics(metrics_df, metric_col='nDCG@10'):
    x = metrics_df[metric_col].values
    w = x / x.sum()
    return dict(zip(metrics_df['model'], w))

base_weights = weights_from_metrics(metrics_df)
print("\nБазовые веса:", base_weights)

# --- 3. Сегментные множители ---
segment_multipliers = {
    "cold":   {"Popularity": 2.0, "Item-Based CF": 0.2, "SVD": 0.2, "Two-Tower": 0.1},
    "warm":   {"Popularity": 0.5, "Item-Based CF": 1.2, "SVD": 0.8, "Two-Tower": 0.6},
    "active": {"Popularity": 0.1, "Item-Based CF": 1.0, "SVD": 0.9, "Two-Tower": 1.2}
}

# --- 4. Гибридная модель ---
class HybridRecommender:
    def __init__(self, models_dict, base_weights, segment_multipliers, popularity_model):
        self.models = models_dict
        self.base_weights = base_weights
        self.segment_multipliers = segment_multipliers
        self.popularity = popularity_model

    def recommend(self, user_id, user_seen_books, k=10):
        segment = get_user_segment(user_id)
        mult = self.segment_multipliers.get(segment, {})
        
        raw_weights = {}
        for name in self.models:
            base_w = self.base_weights.get(name, 0.0)
            multiplier = mult.get(name, 1.0)
            raw_weights[name] = base_w * multiplier
        
        total = sum(raw_weights.values())
        if total == 0:
            weights = {name: 1.0 / len(raw_weights) for name in raw_weights}
        else:
            weights = {name: w / total for name, w in raw_weights.items()}

        all_recs = {}
        for name, model in self.models.items():
            recs = model.recommend(user_id, user_seen_books, k*3)
            for i, book_id in enumerate(recs):
                score = weights[name] * (1.0 / (i + 1))
                all_recs[book_id] = all_recs.get(book_id, 0) + score

        sorted_recs = sorted(all_recs.items(), key=lambda x: x[1], reverse=True)
        final_recs = [book_id for book_id, _ in sorted_recs if book_id not in user_seen_books]
        return final_recs[:k]

models_dict = {
    'Popularity': pop_model,
    'Item-Based CF': item_model,
    'SVD': svd_model,
    'Two-Tower': tt_model
}

hybrid_model = HybridRecommender(
    models_dict=models_dict,
    base_weights=base_weights,
    segment_multipliers=segment_multipliers,
    popularity_model=pop_model
)

# ---Оценка---
SAMPLE_SIZE = 5000
test_users = test['user_id'].unique()
if len(test_users) > SAMPLE_SIZE:
    sampled_users = random.sample(list(test_users), SAMPLE_SIZE)
    test_sampled = test[test['user_id'].isin(sampled_users)].copy()
else:
    test_sampled = test.copy()

start = time.time()
hybrid_metrics = evaluate_metrics(hybrid_model, test_sampled, user_seen_items, all_items, k=10)
hybrid_time = time.time() - start

print("\n--- ФИНАЛЬНЫЙ РЕЗУЛЬТАТ ---")
print(f"Гибридная модель: nDCG@10 = {hybrid_metrics['ndcg@10']:.4f}, время = {hybrid_time:.1f} сек")


Базовые веса: {'Popularity': 0.3003067201111428, 'Item-Based CF': 0.591949262079981, 'SVD': 0.10350162995259325, 'Two-Tower': 0.0042423878562831046}


HybridRecommender: 100%|██████████| 5000/5000 [32:08<00:00,  2.59it/s]  


--- ФИНАЛЬНЫЙ РЕЗУЛЬТАТ ---
Гибридная модель: nDCG@10 = 0.6245, время = 1928.4 сек


In [75]:
# Собираем результаты всех моделей, включая гибрид
all_results = []

# Базовые модели (уже есть в metrics_df)
for _, row in metrics_df.iterrows():
    all_results.append({
        'Модель': row['model'],
        'Precision@10': row['Precision@10'],
        'Recall@10': row['Recall@10'],
        'nDCG@10': row['nDCG@10']
    })

# Добавляем гибридную модель
all_results.append({
    'Модель': 'Hybrid (адаптивный)',
    'Precision@10': hybrid_metrics['precision@10'],
    'Recall@10': hybrid_metrics['recall@10'],
    'nDCG@10': hybrid_metrics['ndcg@10']
})

# Создаём итоговую таблицу
results_df = pd.DataFrame(all_results).set_index('Модель')

# Сортируем по nDCG@10 (по убыванию)
results_df = results_df.sort_values('nDCG@10', ascending=False)

print("\n--- RESULT ---")
print(results_df.round(6))


--- RESULT ---
                     Precision@10  Recall@10   nDCG@10
Модель                                                
Item-Based CF             0.26869   0.126471  0.627471
Hybrid (адаптивный)       0.25458   0.119419  0.624473
Popularity                0.09912   0.044831  0.318328
SVD                       0.02622   0.011746  0.109713
Two-Tower                 0.00103   0.000480  0.004497


Item-Based CF — уже очень сильная модель
Она даёт nDCG = 0.627 — это хороший результат для рекомендательной системы.
Другие модели слабее:
Popularity: nDCG = 0.318 (в 2 раза хуже),
SVD: nDCG = 0.110,
Two-Tower: nDCG = 0.004 (практически бесполезна). Результаты натолкнули  автора на мысль, что, возможно, следует проверить как будет выглядеть вместе CF и Popularity.


## гибрид 2. Item-Based CF + Popularity

In [76]:
models_to_evaluate = [
    ('Popularity', pop_model),
    ('Item-Based CF', item_model)
]

metrics_list_simple = []
for name, model in models_to_evaluate:
    # Используем тот же сэмпл, что и раньше (test_sampled)
    metrics = evaluate_metrics(model, test_sampled, user_seen_items, all_items, k=10)
    metrics_list_simple.append({
        'model': name,
        'Precision@10': metrics['precision@10'],
        'Recall@10': metrics['recall@10'],
        'nDCG@10': metrics['ndcg@10']
    })

metrics_df_simple = pd.DataFrame(metrics_list_simple)
print("Метрики базовых моделей (только CF + Popularity):")
print(metrics_df_simple)

# --- 2. Автоматические веса из метрик ---
base_weights_simple = weights_from_metrics(metrics_df_simple, metric_col='nDCG@10')
print("\nБазовые веса:", base_weights_simple)

# --- 3. Адаптивные сегментные множители ---
segment_multipliers_simple = {
    "cold":   {"Popularity": 2.0, "Item-Based CF": 0.1},    # Акцент на популярность
    "warm":   {"Popularity": 0.3, "Item-Based CF": 1.0},    # Баланс
    "active": {"Popularity": 0.05, "Item-Based CF": 1.0}    # Почти только CF
}

# --- 4. Новый гибридный рекомендатель ---
class SimpleHybridRecommender:
    def __init__(self, models_dict, base_weights, segment_multipliers, popularity_model):
        self.models = models_dict
        self.base_weights = base_weights
        self.segment_multipliers = segment_multipliers
        self.popularity = popularity_model

    def recommend(self, user_id, user_seen_books, k=10):
        segment = get_user_segment(user_id)
        mult = self.segment_multipliers.get(segment, {})
        
        # Рассчитываем финальные веса
        raw_weights = {}
        for name in self.models:
            base_w = self.base_weights.get(name, 0.0)
            multiplier = mult.get(name, 1.0)
            raw_weights[name] = base_w * multiplier
        
        total = sum(raw_weights.values())
        if total == 0:
            weights = {name: 1.0 / len(raw_weights) for name in raw_weights}
        else:
            weights = {name: w / total for name, w in raw_weights.items()}

        # Собираем рекомендации
        all_recs = {}
        for name, model in self.models.items():
            recs = model.recommend(user_id, user_seen_books, k*3)
            for i, book_id in enumerate(recs):
                score = weights[name] * (1.0 / (i + 1))
                all_recs[book_id] = all_recs.get(book_id, 0) + score

        # Сортируем и фильтруем
        sorted_recs = sorted(all_recs.items(), key=lambda x: x[1], reverse=True)
        final_recs = [book_id for book_id, _ in sorted_recs if book_id not in user_seen_books]
        return final_recs[:k]

# --- 5. Создание модели ---
models_dict_simple = {
    'Popularity': pop_model,
    'Item-Based CF': item_model
}

simple_hybrid = SimpleHybridRecommender(
    models_dict=models_dict_simple,
    base_weights=base_weights_simple,
    segment_multipliers=segment_multipliers_simple,
    popularity_model=pop_model
)

# --- 6. Оценка нового гибрида ---
print("\nОценка упрощённого гибрида (CF + Popularity)...")
start = time.time()
simple_hybrid_metrics = evaluate_metrics(simple_hybrid, test_sampled, user_seen_items, all_items, k=10)
simple_time = time.time() - start

print("\n--- РЕЗУЛЬТАТ УПРОЩЁННОГО ГИБРИДА ---")
print(f"nDCG@10 = {simple_hybrid_metrics['ndcg@10']:.6f}")
print(f"Precision@10 = {simple_hybrid_metrics['precision@10']:.6f}")
print(f"Recall@10 = {simple_hybrid_metrics['recall@10']:.6f}")
print(f"Время оценки: {simple_time:.1f} сек")

ItemBasedRecommender: 100%|██████████| 5000/5000 [04:11<00:00, 19.92it/s]


Метрики базовых моделей (только CF + Popularity):
           model  Precision@10  Recall@10   nDCG@10
0     Popularity       0.10344   0.046208  0.323750
1  Item-Based CF       0.27124   0.127021  0.629778

Базовые веса: {'Popularity': 0.33952875637754365, 'Item-Based CF': 0.6604712436224563}

Оценка упрощённого гибрида (CF + Popularity)...


SimpleHybridRecommender: 100%|██████████| 5000/5000 [04:17<00:00, 19.40it/s]


--- РЕЗУЛЬТАТ УПРОЩЁННОГО ГИБРИДА ---
nDCG@10 = 0.629543
Precision@10 = 0.270780
Recall@10 = 0.126832
Время оценки: 257.8 сек


In [ ]:

comparison_data = {
    'Модель': ['Item-Based CF', 'Упрощённый гибрид'],
    'nDCG@10': [0.627471, 0.629543],
    'Precision@10': [0.268690, 0.270780],
    'Recall@10': [0.126471, 0.126832],
    'Время (сек)': [300, 257.8]
}

df_comparison = pd.DataFrame(comparison_data).set_index('Модель')

print("\n--- СРАВНЕНИЕ МОДЕЛЕЙ ---")
display(df_comparison.style.format({
    'nDCG@10': '{:.6f}',
    'Precision@10': '{:.6f}',
    'Recall@10': '{:.6f}',
    'Время (сек)': '{:.1f}'
}))


--- СРАВНЕНИЕ МОДЕЛЕЙ ---


,nDCG@10,Precision@10,Recall@10,Время (сек)
Модель,,,,
Item-Based CF,0.627471,0.268690,0.126471,300.0
Упрощённый гибрид,0.629543,0.270780,0.126832,257.8


**Итак, удалось превзойти "чистую" модель лидер CF**. 



Теперь в гибриде **только две сильные и дополняющие модели**:

Item-Based CF — для персонализации,
Popularity — для стабильности и cold-start. 

 Адаптивные веса работают:

* Для активных пользователей: вес CF ≈ 95%, Popularity ≈ 5% → почти чистый CF.

* Для новых пользователей: вес Popularity ≈ 95% → надёжные рекомендации.
* В среднем — система умнее балансирует, чем ручное взвешивание.

Ранговое объединение усиливает лучшие рекомендации:

Если и CF, и Popularity рекомендуют одну и ту же популярную книгу — её скор суммируется → она поднимается выше. Это особенно помогает в верхней части топ-10, что прямо влияет на nDCG.



## Выводы

## Сравнительный Анализ Моделей

В рамках проекта были реализованы и оценены четыре рекомендательные модели, а также их гибридная комбинация. Все модели оценивались по трём ключевым метрикам на репрезентативной выборке из 5000 пользователей:

- Precision@10 — доля релевантных книг в топ-10 рекомендаций,
- Recall@10 — доля всех релевантных книг пользователя, попавших в топ-10,
- nDCG@10 — метрика ранжирования, учитывающая позицию релевантных книг (чем выше — тем лучше).

Результаты представлены в таблице ниже:

| Модель             | Precision@10 | Recall@10 | nDCG@10 |
|--------------------|--------------|-----------|---------|
| Item-Based CF      | 0.268690     | 0.126471  | 0.627471|
| Hybrid (CF + Popularity) | 0.270780 | 0.126832  | 0.629543|
| Popularity         | 0.099120     | 0.044831  | 0.318328|
| SVD                | 0.026220     | 0.011746  | 0.109713|
| Two-Tower          | 0.001030     | 0.000480  | 0.004497|

### Интерпретация результатов

#### Item-Based Collaborative Filtering — сильная базовая модель 

  Модель использует фактическую историю оценок для построения матрицы схожести между книгами. Это позволяет точно улавливать тематические и стилистические предпочтения пользователя, особенно если у него длинная история взаимодействий.
- Преимущества:  
  Высокая персонализация, интерпретируемость, устойчивость к шуму в данных.
- Недостатки:  
  Страдает от проблемы холодного старта — не может рекомендовать книги новым пользователям или книгам без оценок.

#### Гибридная модель (Item-Based CF + Popularity) — фаворит
 
  Гибрид сохраняет силу персонализации Item-Based CF для активных пользователей, но добавляет устойчивость за счёт Popularity для новых и малоактивных пользователей. Автоматическое взвешивание на основе метрик и сегментация позволяют гибко адаптироваться к разным типам пользователей.
- Результат:  
  Небольшой, но статистически значимый прирост по всем метрикам — nDCG@10 = 0.6295 (лучший результат среди всех моделей).

#### Popularity — надёжный baseline
  
  Рекомендует самые популярные книги, что даёт стабильные, но неперсонализированные результаты. Эффективна для новых пользователей, но бесполезна для опытных.
- Роль в системе:  
  Ключевой компонент гибридной модели для обработки cold-start.

#### SVD — слабый результат

Как мы выяснили на этапе бейзлайна, тут присутствует разреженность данных.
SVD пытается восстановить эту матрицу через низкоранговое приближение, но при такой разреженности:
Векторы пользователей/книг плохо обучаются,
Модель не может выявить скрытые паттерны
 
#### Two-Tower (нейросеть) — неэффективна на данном датасете при данном количестве ресурсов
- Почему провалилась:  
  Нейросетевая модель показала низкие метрики. Скорее всего по причине небольшого количества эпох. На самом деле, после создания расширенных признаков именно от этой этой модели ожидался результат. Однако ж, она очень долго учится и нормально ее обучить, к сожалению, не вышло. И, получается, хорошо подобранными классическими моделями, в виде гибрида, можно получить хорошие метрики. И, возможно, именно использование башен нецелесообразно при ограниченном ресурсе.
.

### Общий вывод

- Item-Based CF оказалась наиболее эффективной классической моделью, что типично для датасетов с богатой историей оценок и явными тематическими связями (как в Goodreads).
- Гибридная система на её основе достигла максимального качества, продемонстрировав силу комбинированного подхода.
- Нейросетевые методы (Two-Tower) в данном случае не оправдали ожиданий, что подчёркивает важность выбора метода под специфику данных: иногда простые и интерпретируемые модели работают лучше сложных black-box решений.

Таким образом, финальная рекомендательная система, основанная на гибриде Item-Based CF и Popularity, является оптимальным балансом качества, скорости и устойчивости для поставленной задачи.